# For figure 6.25

Temperature response to forcing 2020-2100

In [1]:
import numpy as np
import scipy.stats as st
import pandas as pd
import matplotlib.pyplot as pl
import os
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
import matplotlib.patches as mp
from netCDF4 import Dataset
import warnings

from ar6.utils.h5 import *

In [2]:
# TODO - sort out figure sizing

pl.rcParams['figure.figsize'] = (18/2.54, 11/2.54)
pl.rcParams['font.size'] = 11
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'out'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.major.left'] = True
pl.rcParams['ytick.major.size'] = 0
pl.rcParams['xtick.top'] = True
pl.rcParams['figure.dpi'] = 150

In [3]:
results = load_dict_from_hdf5('../data_output_large/twolayer_SSPs.h5')

In [4]:
results.keys()

dict_keys(['ssp119', 'ssp119_climuncert', 'ssp119_remove_aerosol', 'ssp119_remove_aerosol_climuncert', 'ssp119_remove_bc_on_snow', 'ssp119_remove_bc_on_snow_climuncert', 'ssp119_remove_ch4', 'ssp119_remove_ch4_climuncert', 'ssp119_remove_co2', 'ssp119_remove_co2_climuncert', 'ssp119_remove_hfc', 'ssp119_remove_hfc_climuncert', 'ssp119_remove_o3', 'ssp119_remove_o3_climuncert', 'ssp119_remove_other_anthro', 'ssp119_remove_other_anthro_climuncert', 'ssp119_remove_other_ghg', 'ssp119_remove_other_ghg_climuncert', 'ssp119_remove_slcf', 'ssp119_remove_slcf_climuncert', 'ssp126', 'ssp126_climuncert', 'ssp126_remove_aerosol', 'ssp126_remove_aerosol_climuncert', 'ssp126_remove_bc_on_snow', 'ssp126_remove_bc_on_snow_climuncert', 'ssp126_remove_ch4', 'ssp126_remove_ch4_climuncert', 'ssp126_remove_co2', 'ssp126_remove_co2_climuncert', 'ssp126_remove_hfc', 'ssp126_remove_hfc_climuncert', 'ssp126_remove_o3', 'ssp126_remove_o3_climuncert', 'ssp126_remove_other_anthro', 'ssp126_remove_other_anthro_cl

In [5]:
results['ssp245']['surface_temperature'].shape

(351, 2237)

In [6]:
results['ssp245']['surface_temperature'][269].mean()

1.288290462454323

In [7]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']
scenarios = ['ssp119','ssp126','ssp245','ssp370','ssp370-lowNTCF-aerchemmip','ssp370-lowNTCF-gidden','ssp434','ssp460','ssp534-over','ssp585']
for scenario in scenarios:
    for forcing in forcings:
        print(scenario, forcing, (results[scenario]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351]).mean())

ssp119 ch4 0.1479647406366074
ssp119 hfc 0.012881854080715283
ssp119 o3 0.07239981177736933
ssp119 aerosol -0.16204737745288333
ssp119 bc_on_snow 0.006492640422369413
ssp119 slcf 0.0776916694641756
ssp126 ch4 0.15700660225582758
ssp126 hfc 0.015565393942750007
ssp126 o3 0.08150177214863914
ssp126 aerosol -0.1633076405786558
ssp126 bc_on_snow 0.009652568249125257
ssp126 slcf 0.1004186960179692
ssp245 ch4 0.28489547465866527
ssp245 hfc 0.06409582612714346
ssp245 o3 0.17564044531806106
ssp245 aerosol -0.27867451649624725
ssp245 bc_on_snow 0.016803998145734994
ssp245 slcf 0.262761227753683
ssp370 ch4 0.49814623265246455
ssp370 hfc 0.09202715890303328
ssp370 o3 0.39550986346283395
ssp370 aerosol -0.5732443586632884
ssp370 bc_on_snow 0.063835775125474
ssp370 slcf 0.47627467148086877
ssp370-lowNTCF-aerchemmip ch4 0.4981462326526299
ssp370-lowNTCF-aerchemmip hfc 0.09202715890335239
ssp370-lowNTCF-aerchemmip o3 0.27354484100678517
ssp370-lowNTCF-aerchemmip aerosol -0.28642880522282643
ssp370-lo

In [8]:
(results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][331:351] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][0]).shape

(20, 2237)

In [9]:
forcings = ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf']

AR6_forc = {}
AR6_ecsforc = {}

base_periods = {
#    '1750': 0,
#    '1850-1900': slice(100,151),
#    '1995-2014': slice(245,265),
#    '2010-2019': slice(260,270),
    '2020'     : slice(270,271)
}

for scenario in scenarios:
    AR6_forc[scenario] = {}
    AR6_ecsforc[scenario] = {}
    for forcing in forcings:
        AR6_forc[scenario][forcing] = {}
        AR6_ecsforc[scenario][forcing] = {}
        for base_period in base_periods:
            AR6_forc[scenario][forcing][base_period] = np.zeros((81,5))
            AR6_ecsforc[scenario][forcing][base_period] = np.zeros((81,5))
            for year in range(270,351):
                AR6_forc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results[scenario]['surface_temperature'][year]- results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
                AR6_ecsforc[scenario][forcing][base_period][year-270,:] = np.percentile(
                    (
                        (results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0))-
                        (results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][year] - results['%s_remove_%s_climuncert' % (scenario, forcing)]['surface_temperature'][base_periods[base_period]].mean(axis=0))
                    ), (5,16,50,84,95)
                )
    AR6_forc[scenario]['anthro'] = {}
    AR6_ecsforc[scenario]['anthro'] = {}
    for base_period in base_periods:
        AR6_forc[scenario]['anthro'][base_period] = np.zeros((81,5))
        AR6_ecsforc[scenario]['anthro'][base_period] = np.zeros((81,5))
        for year in range(270,351):
            AR6_forc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results[scenario]['surface_temperature'][year] - results[scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))
            AR6_ecsforc[scenario]['anthro'][base_period][year-270,:] = np.percentile(results['%s_climuncert' % scenario]['surface_temperature'][year] - results['%s_climuncert' % scenario]['surface_temperature'][base_periods[base_period]].mean(axis=0), (5,16,50,84,95))

In [10]:
print(AR6_ecsforc['ssp245']['anthro']['2020'])

[[0.         0.         0.         0.         0.        ]
 [0.02029008 0.02334659 0.02917689 0.0362141  0.04100446]
 [0.04022997 0.04595305 0.05705097 0.07131904 0.08042815]
 [0.05977569 0.0681605  0.08421634 0.10506646 0.11849794]
 [0.0788883  0.08953525 0.11082231 0.13803681 0.15562157]
 [0.09734282 0.11096412 0.13683337 0.17008398 0.19217519]
 [0.11505448 0.13177936 0.16240246 0.20176629 0.22809539]
 [0.13280222 0.15193979 0.18777451 0.23280916 0.26377121]
 [0.14996364 0.17205725 0.2129499  0.26364653 0.29877825]
 [0.16735723 0.19204257 0.23787929 0.29421252 0.33381353]
 [0.18506933 0.21145561 0.26185945 0.32516147 0.36692739]
 [0.20214157 0.2314656  0.2861229  0.35566101 0.4022105 ]
 [0.21948507 0.25162626 0.31069496 0.38737499 0.43771287]
 [0.23665119 0.27169307 0.3356457  0.41921293 0.47393044]
 [0.25428617 0.29208578 0.36078842 0.45127709 0.51004752]
 [0.27173711 0.3121593  0.38602737 0.48308099 0.54727041]
 [0.28893093 0.3325208  0.41105582 0.51530466 0.58337812]
 [0.30590407 0

In [11]:
for scenario in scenarios:
    print(scenario, AR6_ecsforc[scenario]['anthro']['2020'][:,[0,2,4]])

ssp119 [[ 0.          0.          0.        ]
 [ 0.01863511  0.02933652  0.04376437]
 [ 0.0357448   0.0577557   0.08803244]
 [ 0.05104338  0.08526933  0.1322487 ]
 [ 0.06487676  0.11168563  0.175732  ]
 [ 0.07740503  0.13686435  0.21746543]
 [ 0.08819101  0.16036839  0.25989752]
 [ 0.09747427  0.1825694   0.29911908]
 [ 0.1047958   0.20270141  0.33801692]
 [ 0.11070545  0.2219653   0.3766068 ]
 [ 0.11538175  0.23918647  0.41323123]
 [ 0.11910046  0.25413283  0.44732219]
 [ 0.12087065  0.26611112  0.47731329]
 [ 0.1229057   0.27515135  0.50191024]
 [ 0.12347329  0.28255564  0.52122035]
 [ 0.12339111  0.28879943  0.53874629]
 [ 0.12234144  0.29320599  0.55422616]
 [ 0.12129657  0.29685068  0.56606196]
 [ 0.12018403  0.29830349  0.57457431]
 [ 0.1186757   0.29945088  0.58325663]
 [ 0.11724449  0.30024159  0.59117922]
 [ 0.11473355  0.29991825  0.59713294]
 [ 0.11146572  0.29941208  0.60211466]
 [ 0.10696353  0.29810273  0.60663478]
 [ 0.10246246  0.29700695  0.61033183]
 [ 0.09881966  0.2

In [12]:
output = []
for scenario in scenarios:
    for forcing in ['ch4', 'hfc', 'o3', 'aerosol', 'bc_on_snow', 'slcf', 'anthro']:
        for year in range(270, 351):
            output.append([scenario, forcing, '2020', year+1750,
               AR6_ecsforc[scenario][forcing]['2020'][year-270,0],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,1],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,2],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,3],
               AR6_ecsforc[scenario][forcing]['2020'][year-270,4],
            ])

df = pd.DataFrame(output, columns=['scenario', 'forcing', 'base_period', 'year', 'p05','p16','p50','p84','p95'])
df.to_csv('../data_output/slcf_warming_ranges.csv')
df

,scenario,forcing,base_period,year,p05,p16,p50,p84,p95
0,ssp119,ch4,2020,2020,0.000000,0.000000,0.000000,0.000000,0.000000
1,ssp119,ch4,2020,2021,0.001266,0.001475,0.001859,0.002378,0.002767
2,ssp119,ch4,2020,2022,0.002245,0.002638,0.003392,0.004398,0.005168
3,ssp119,ch4,2020,2023,0.002916,0.003463,0.004565,0.006049,0.007151
4,ssp119,ch4,2020,2024,0.003252,0.003953,0.005340,0.007207,0.008723
...,...,...,...,...,...,...,...,...,...
5665,ssp585,anthro,2020,2096,2.437122,2.772979,3.416691,4.215365,4.876034
5666,ssp585,anthro,2020,2097,2.467945,2.808497,3.460478,4.273505,4.942096
5667,ssp585,anthro,2020,2098,2.497630,2.842573,3.503622,4.330887,5.006969
5668,ssp585,anthro,2020,2099,2.526265,2.877141,3.547548,4.387970,5.070724
